# Chapter 1 Environmental Engineering: Air Quality Modeling

1. [Introduction: BOD equation](#1.-Introduction)
2. [Simulation: BOD](#2.-Simulation)
3. [Self-Assessment](#3.-Self-Assessment)

## 1. Introduction

### Methodology: Watershed Runoff, Sediment, and Nutrient Simulation

This simulation estimates hydrologic and water quality responses of a watershed under different land use scenarios. It integrates rainfall, land use type, slope, area, and drainage density to compute:

- Surface runoff (mm and m³)
- Sediment yield (tons)
- Nutrient loads (Nitrogen and Phosphorus in kg)
- Change analysis between baseline and future land use

---

### 1. 🌧️ Runoff Estimation: NRCS Curve Number Method

Runoff is calculated using the **NRCS-CN method**, which relates land use and soil conditions to rainfall retention:

$$
S = \frac{25400}{CN} - 254
\quad ; \quad
I_a = 0.2 \cdot S
\quad ; \quad
Q = \frac{(P - I_a)^2}{P + 0.8 \cdot S}
$$

Where:
- $( CN $): Curve Number (based on land use)
- $( P $): Rainfall (mm)
- $( Q $): Runoff depth (mm)

Runoff volume is then computed as:

$$
Q_{\text{vol}} = Q \cdot \frac{\text{Area}_{\text{ha}} \cdot 10,000}{1000}
$$

---

### 2. Sediment Yield: Modified Universal Soil Loss Equation (MUSLE)

Sediment yield is estimated using a simplified form of **MUSLE**:

$$
S = 11.8 \cdot Q \cdot A \cdot K \cdot C \cdot P \cdot LS
$$

Where:
- $( Q $): Runoff (mm)
- $( A $): Area (ha)
- $( K $): Soil erodibility factor (default 0.3)
- $( C $): Cover factor (default 0.2)
- $( P $): Practice factor (default 1.0)
- $( LS $): Slope length and steepness factor (approximated as slope % / 100)

---

### 🧪 Nutrient Load Estimation

Nutrient loads are estimated using **land use-specific export coefficients

In [2]:
# 📌 Run this cell in a Jupyter Notebook
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# 📐 Curve Number lookup
def get_curve_number(landuse):
    cn_dict = {
        'Forest': 55,
        'Urban': 85,
        'Agriculture': 75,
        'Wetland': 50,
        'Grassland': 60
    }
    return cn_dict.get(landuse, 70)

# 📐 Nutrient export coefficients (kg/ha/year)
def get_nutrient_coeffs(landuse):
    coeffs = {
        'Forest': {'N': 1.0, 'P': 0.1},
        'Urban': {'N': 5.0, 'P': 1.0},
        'Agriculture': {'N': 15.0, 'P': 2.5},
        'Wetland': {'N': 0.5, 'P': 0.05},
        'Grassland': {'N': 3.0, 'P': 0.5}
    }
    return coeffs.get(landuse, {'N': 5.0, 'P': 1.0})

# 📐 Runoff estimation using NRCS-CN method
def estimate_runoff(rainfall_mm, CN):
    S = (25400 / CN) - 254
    Ia = 0.2 * S
    if rainfall_mm <= Ia:
        return 0
    Q = ((rainfall_mm - Ia)**2) / (rainfall_mm + 0.8 * S)
    return round(Q, 2)

# 📐 Sediment yield using MUSLE (simplified)
def estimate_sediment(Q, area_ha, slope_percent, K=0.3, C=0.2, P=1.0):
    LS = slope_percent / 100
    sediment_tons = 11.8 * Q * area_ha * K * C * P * LS
    return round(sediment_tons, 2)

# 📐 Nutrient loads
def estimate_nutrients(area_ha, landuse):
    coeffs = get_nutrient_coeffs(landuse)
    N_load = coeffs['N'] * area_ha
    P_load = coeffs['P'] * area_ha
    return round(N_load, 2), round(P_load, 2)

# 📊 Simulation function
def watershed_comparison(rainfall_mm, landuse_base, landuse_future, area_km2, slope_percent, drainage_density):
    clear_output(wait=True)

    area_ha = area_km2 * 100

    def simulate(landuse):
        CN = get_curve_number(landuse)
        runoff_mm = estimate_runoff(rainfall_mm, CN)
        runoff_m3 = runoff_mm / 1000 * area_ha * 10000
        sediment = estimate_sediment(runoff_mm, area_ha, slope_percent)
        N_load, P_load = estimate_nutrients(area_ha, landuse)
        return runoff_m3, sediment, N_load, P_load

    base = simulate(landuse_base)
    future = simulate(landuse_future)

    # 📋 Output summary
    print(f"🌧️ Rainfall: {rainfall_mm:.1f} mm")
    print(f"📐 Area: {area_km2:.2f} km² | Slope: {slope_percent:.1f}% | Drainage Density: {drainage_density:.2f} km/km²\n")

    print(f"🔄 Baseline Land Use: {landuse_base}")
    print(f"💧 Runoff: {base[0]:,.0f} m³ | Sediment: {base[1]:.2f} tons | N: {base[2]:.2f} kg | P: {base[3]:.2f} kg")

    print(f"\n🔁 Future Land Use: {landuse_future}")
    print(f"💧 Runoff: {future[0]:,.0f} m³ | Sediment: {future[1]:.2f} tons | N: {future[2]:.2f} kg | P: {future[3]:.2f} kg")

    print("\n📊 Change Summary:")
    labels = ['Runoff (m³)', 'Sediment (tons)', 'Nitrogen (kg)', 'Phosphorus (kg)']
    for i, label in enumerate(labels):
        delta = future[i] - base[i]
        pct = (delta / base[i] * 100) if base[i] != 0 else 0
        print(f" - {label}: Δ = {delta:.2f} ({pct:+.1f}%)")

    # 📈 Visualization
    x = np.arange(len(labels))
    width = 0.35
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(x - width/2, base, width, label='Baseline', color='lightblue', edgecolor='black')
    ax.bar(x + width/2, future, width, label='Future', color='salmon', edgecolor='black')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.set_title('Watershed Output Comparison')
    ax.set_ylabel('Magnitude')
    ax.legend()
    ax.grid(True, axis='y')
    plt.tight_layout()
    plt.show()

# 🎚️ Interactive controls
rainfall_slider = widgets.FloatSlider(value=50.0, min=10.0, max=200.0, step=5.0, description='Rainfall (mm)')
landuse_base_dropdown = widgets.Dropdown(options=['Forest', 'Urban', 'Agriculture', 'Wetland', 'Grassland'], value='Agriculture', description='Baseline Land Use')
landuse_future_dropdown = widgets.Dropdown(options=['Forest', 'Urban', 'Agriculture', 'Wetland', 'Grassland'], value='Urban', description='Future Land Use')
area_slider = widgets.FloatSlider(value=2.0, min=0.1, max=50.0, step=0.1, description='Area (km²)')
slope_slider = widgets.FloatSlider(value=5.0, min=0.1, max=30.0, step=0.5, description='Slope (%)')
drainage_slider = widgets.FloatSlider(value=1.5, min=0.5, max=5.0, step=0.1, description='Drainage Density')

interactive_plot = widgets.interactive(
    watershed_comparison,
    rainfall_mm=rainfall_slider,
    landuse_base=landuse_base_dropdown,
    landuse_future=landuse_future_dropdown,
    area_km2=area_slider,
    slope_percent=slope_slider,
    drainage_density=drainage_slider
)

display(interactive_plot)

interactive(children=(FloatSlider(value=50.0, description='Rainfall (mm)', max=200.0, min=10.0, step=5.0), Dro…

###  Watershed Simulation: Quiz, Conceptual & Reflective Questions

This module reinforces understanding of watershed runoff, sediment yield, and nutrient loading using NRCS-CN, MUSLE, and export coefficient methods. It supports learning through multiple-choice questions, conceptual prompts, and reflective challenges.

---

###  Conceptual Questions

1. **What does the Curve Number (CN) primarily represent?**
   - A. Soil erosion potential  
   - B. Land use and soil infiltration characteristics  
   - C. Nutrient export rate  
   - D. Slope steepness  

2. **Which factor most directly increases sediment yield in the MUSLE equation?**
   - A. Rainfall intensity  
   - B. Slope percentage  
   - C. Drainage density  
   - D. Phosphorus coefficient  

3. **What is the role of the initial abstraction \( I_a \) in the NRCS-CN method?**
   - A. It represents the maximum runoff  
   - B. It accounts for infiltration and surface storage before runoff begins  
   - C. It adjusts nutrient loading  
   - D. It scales sediment transport  

4. **Which land use type typically has the highest nitrogen export coefficient?**
   - A. Forest  
   - B. Urban  
   - C. Agriculture  
   - D. Wetland  

5. **What does the LS factor in MUSLE represent?**
   - A. Land use and slope  
   - B. Length and steepness of slope  
   - C. Load and sediment transport  
   - D. Land surface roughness  

---

###  Interpretation

1. Why is runoff volume converted from mm to cubic meters using area and unit conversions?  
2. How does changing slope percentage affect sediment yield in the MUSLE calculation?  
3. Why are nutrient loads estimated using static coefficients rather than dynamic concentration models?  
4. What assumptions are made when comparing baseline and future land use scenarios?  
5. How could drainage density be incorporated more directly into sediment or runoff estimation?

---

###  Reflective Questions

1. **How do land use changes influence both water quantity and quality in a watershed?**  
2. **Why is it important to simulate both runoff and sediment/nutrient loads when planning restoration?**  
3. **What are the limitations of using empirical coefficients for nutrient export?**  
4. **How could this model be adapted to include BMPs (e.g., buffer strips, retention ponds)?**  
5. **What insights can be gained by visualizing the percent change between baseline and future scenarios?**

---